# Introduction to Amazon Rekognition

This notebook uploads the `images/` content into Amazon S3 for processing with Rekognition.

First, you'll need to install the AWS SDK for Python-- [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html). This module generates our service clients and simplifies accessing AWS APIs.

In [5]:
!pip3 install boto3

You should consider upgrading via the 'c:\tools\python3.8\python.exe -m pip install --upgrade pip' command.


Next, configure this notebooks global settings. 

1. Set the `bucket_name` variable to an S3 bucket within your AWS account
1. Set the `region_name` to the S3 bucket's region
1. Create the `rekognition` client for the same region

In [6]:
import boto3 

bucket_name = 'cv-on-aws-book-nbachmei'
region_name = 'us-east-2'
rekognition = boto3.client('rekognition', region_name=region_name)

Now, let's invoke the `DetectLabels` API to inspect the `images/skateboard.jpg` image.

In [29]:
response = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': 'chapter_02/images/skateboard.jpg'
        }
    })

print(response)

{'Labels': [{'Name': 'Car', 'Confidence': 99.15271759033203, 'Instances': [{'BoundingBox': {'Width': 0.10616335272789001, 'Height': 0.18528179824352264, 'Left': 0.0037978871259838343, 'Top': 0.5039216876029968}, 'Confidence': 99.15271759033203}, {'BoundingBox': {'Width': 0.2429988533258438, 'Height': 0.21577216684818268, 'Left': 0.7309805154800415, 'Top': 0.5251884460449219}, 'Confidence': 99.1286392211914}, {'BoundingBox': {'Width': 0.14233611524105072, 'Height': 0.15528248250484467, 'Left': 0.6494812965393066, 'Top': 0.5333095788955688}, 'Confidence': 98.48368072509766}, {'BoundingBox': {'Width': 0.11086395382881165, 'Height': 0.10271988064050674, 'Left': 0.10355594009160995, 'Top': 0.5354844927787781}, 'Confidence': 96.45606231689453}, {'BoundingBox': {'Width': 0.06254628300666809, 'Height': 0.053911514580249786, 'Left': 0.46083059906959534, 'Top': 0.5573825240135193}, 'Confidence': 93.65448760986328}, {'BoundingBox': {'Width': 0.10105428099632263, 'Height': 0.12226245552301407, 'Le

Next, let's persist this output into a JSON-formatted document.

In [8]:
from json import dumps
with open('response.json', 'w') as f:
    f.write(dumps(response, indent=2))

You probably noticed that several labels have a low `confidence` score. Let's filter those out and find the top-10 best labels.

In [9]:
from json import dumps
filtered_response = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': 'chapter_02/images/skateboard.jpg'
        },
    },
    MinConfidence= 80,
    MaxLabels= 10
    )

print(dumps(filtered_response, indent=2))

{
  "Labels": [
    {
      "Name": "Car",
      "Confidence": 99.15271759033203,
      "Instances": [
        {
          "BoundingBox": {
            "Width": 0.10616335272789001,
            "Height": 0.18528179824352264,
            "Left": 0.0037978871259838343,
            "Top": 0.5039216876029968
          },
          "Confidence": 99.15271759033203
        },
        {
          "BoundingBox": {
            "Width": 0.2429988533258438,
            "Height": 0.21577216684818268,
            "Left": 0.7309805154800415,
            "Top": 0.5251884460449219
          },
          "Confidence": 99.1286392211914
        },
        {
          "BoundingBox": {
            "Width": 0.14233611524105072,
            "Height": 0.15528248250484467,
            "Left": 0.6494812965393066,
            "Top": 0.5333095788955688
          },
          "Confidence": 98.48368072509766
        },
        {
          "BoundingBox": {
            "Width": 0.11086395382881165,
            "Height

The `DetectLabel` API returns a set of hierarchical labels. Let's use the `treelib` module to render taxonomy. 

In [10]:
from typing import Mapping, List
from json import dumps
from treelib import Node, Tree

labels:Mapping[str,List[str]] = {}
for label in response['Labels']:
    parents = [x['Name'] for x in label['Parents']]
    name = label['Name']

    if len(parents) == 0:
        labels[name] = []
    else:
        for parent in parents:
            if parent in labels:
                labels[parent].append(name)
            else:
                labels[parent] = [name]

tree = Tree()
root = tree.create_node('Response')
for [label,children] in labels.items():
    node = tree.create_node(label, parent=root.identifier)
    for child in children:
        tree.create_node(child, parent=node.identifier)

tree.show()

Response
├── Asphalt
├── Building
│   ├── Downtown
│   └── Neighborhood
├── Car
│   ├── Coupe
│   ├── Parking
│   ├── Parking Lot
│   └── Sports Car
├── City
│   └── Downtown
├── Human
├── Machine
├── Path
│   ├── Pavement
│   └── Sidewalk
├── Person
│   ├── Pedestrian
│   ├── Skateboard
│   ├── Sport
│   └── Sports
├── Road
│   └── Intersection
├── Sport
│   └── Skateboard
├── Sports Car
│   └── Coupe
├── Tarmac
├── Transportation
│   ├── Coupe
│   ├── Parking
│   ├── Parking Lot
│   └── Sports Car
├── Urban
│   ├── Downtown
│   └── Neighborhood
└── Vehicle
    ├── Automobile
    ├── Car
    ├── Coupe
    ├── Parking
    ├── Parking Lot
    └── Sports Car



Let's draw a rectangle around the `Skateboard`. This task requires first finding the label's `BoundingBox` information.

In [11]:
from json import dumps

skateboards = []
for label in response['Labels']:
    if label['Name'] == 'Skateboard':
        skateboards.append(label)

print(dumps(skateboards, indent=2))

[
  {
    "Name": "Skateboard",
    "Confidence": 92.37877655029297,
    "Instances": [
      {
        "BoundingBox": {
          "Width": 0.12326609343290329,
          "Height": 0.058117982000112534,
          "Left": 0.44815489649772644,
          "Top": 0.6332163214683533
        },
        "Confidence": 92.37877655029297
      }
    ],
    "Parents": [
      {
        "Name": "Sport"
      },
      {
        "Name": "Person"
      }
    ]
  }
]


The `treelib` module provides a simple API for creating our trees.

In [12]:
from treelib import Tree, Node

myTree = Tree()
transportation_node = myTree.create_node('Transportation')
vehicle_node = myTree.create_node('Vehicle', parent=transportation_node.identifier)
car_node = myTree.create_node('Car', parent=vehicle_node.identifier)
sedan_node = myTree.create_node('Sedan', parent=car_node.identifier)

myTree.show()

Transportation
└── Vehicle
    └── Car
        └── Sedan



Let's use the `treelib` module to render the response from the `DetectLabel` API.

In [15]:
from treelib import Tree, Node

another_tree = Tree()
root = another_tree.create_node('Response')
model_version_node = another_tree.create_node(
    'LabelModelVersion : %s' % response['LabelModelVersion'],
    parent=root.identifier)
labels_node = another_tree.create_node(
    'Labels: %d total' % len(response['Labels']),
    parent=root.identifier)

sorted_by_confidence = sorted(
    response['Labels'],
    key=lambda label: label['Confidence'], 
    reverse=True)

top_10 = sorted_by_confidence[:10]

for label in top_10:
    name = label['Name']    
    confidence = label['Confidence']
    label_node = another_tree.create_node(
        '%15s - [Confidence: %2.2f%%]' % (name, confidence),
        parent=labels_node.identifier)

another_tree.show()

Response
├── LabelModelVersion : 2.0
└── Labels: 29 total
    ├──             Car - [Confidence: 99.15%]
    ├──            Road - [Confidence: 92.82%]
    ├──           Human - [Confidence: 98.99%]
    ├──           Wheel - [Confidence: 93.25%]
    ├──          Person - [Confidence: 98.99%]
    ├──         Machine - [Confidence: 93.25%]
    ├──         Vehicle - [Confidence: 99.15%]
    ├──      Automobile - [Confidence: 99.15%]
    ├──      Pedestrian - [Confidence: 92.78%]
    └──  Transportation - [Confidence: 99.15%]



Let's encapsulate our label search logic into the `find_first_label` function.

In [30]:
from json import dumps

def find_first_label(response, label_name):  
    for label in response['Labels']:
        if label['Name'] == label_name:
            return label

    print('Unable to find a %s label' % label_name)
    return None
   
skateboard = find_first_label(response, 'Skateboard')
print(dumps(skateboard, indent=2))

{
  "Name": "Skateboard",
  "Confidence": 92.37877655029297,
  "Instances": [
    {
      "BoundingBox": {
        "Width": 0.12326609343290329,
        "Height": 0.058117982000112534,
        "Left": 0.44815489649772644,
        "Top": 0.6332163214683533
      },
      "Confidence": 92.37877655029297
    }
  ],
  "Parents": [
    {
      "Name": "Sport"
    },
    {
      "Name": "Person"
    }
  ]
}


Use the Amazon S3 `GetObject` API to download the `skateboard.jpg`

In [20]:
import boto3

s3_client= boto3.client('s3', region_name=region_name)
image_file = s3_client.get_object(
    Bucket= bucket_name,
    Key= 'chapter_02/images/skateboard.jpg')

image_bytes = image_file['Body'].read()


The Python Image Library (PIL) lets you read, write, and mutate image files.

In [19]:
!pip3 install pillow

ERROR: Could not find a version that satisfies the requirement PIL
ERROR: No matching distribution found for PIL
You should consider upgrading via the 'c:\tools\python3.8\python.exe -m pip install --upgrade pip' command.


You can pass the `image_bytes` to the `Image.open` function using the `BytesIO` object. This approach avoids writing the bytes to disk and then reading them back.

In [54]:
from PIL import Image
from io import BytesIO

image = Image.open(BytesIO(image_bytes))
print(image.size)

image.show()

(1200, 800)


Next, let's denormalize the `BoundingBox` and draw it around the skakeboard.

In [55]:
from PIL import Image, ImageDraw

drawing = ImageDraw.Draw(image)
for instance in skateboard['Instances']:
    bounding_box = instance['BoundingBox']
    
    width = int(bounding_box['Width'] * image.size[0])
    left = int(bounding_box['Left'] * image.size[0])

    height = int(bounding_box['Height'] * image.size[1])
    top = int(bounding_box['Top'] * image.size[1])
        
    drawing.rectangle(
        xy=(left,top,left+width, left+height),
        outline='red')

image.show() 

width=147; height=46; top=506; left=537


Several Amazon Rekognition APIs support base64 encoding the image and passing directly to the action. This process lets you skip placing the file in Amazon S3 first. 

In [4]:
import boto3
from base64 import b64encode

rekognition = boto3.client('rekognition')

with open('images/skateboard.jpg','rb') as f:
    image_bytes = b64encode(f.read())

    rekognition.detect_labels(
        Image={
            "Bytes": image_bytes
        })